In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds

In [2]:
import urllib3
urllib3.disable_warnings()

#tfds.disable_progress_bar()   # 이 주석을 풀면 데이터셋 다운로드과정의 프로그레스바가 나타나지 않습니다.

(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    shuffle_files=True,
    with_info=True,
)

In [3]:
def normalize_and_resize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    # image = tf.image.resize(image, [32, 32])
    return tf.cast(image, tf.float32) / 255., label

In [4]:
def apply_normalize_on_dataset(ds, is_test=False, batch_size=16):
    ds = ds.map(
        normalize_and_resize_img, 
        num_parallel_calls=1
    )
    ds = ds.batch(batch_size)
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [5]:
class VGG11(tf.keras.Model):
    def __init__(self):
        super(VGG11,self).__init__()
        self.conv1_1 = tf.keras.layers.Conv2D(64, kernel_size=3, strides=1, padding='same',activation='relu')
        self.maxpool1 = tf.keras.layers.MaxPool2D(pool_size=2)
        
        self.conv2_1 = tf.keras.layers.Conv2D(128, kernel_size=3, strides=1, padding='same',activation='relu')
        self.maxpool2 = tf.keras.layers.MaxPool2D(pool_size=2)
        
        self.conv3_1 = tf.keras.layers.Conv2D(256, kernel_size=3, strides=1, padding='same',activation='relu')
        self.conv3_2 = tf.keras.layers.Conv2D(256, kernel_size=3, strides=1, padding='same',activation='relu')
        self.maxpool3 = tf.keras.layers.MaxPool2D(pool_size=2)
        
        self.conv4_1 = tf.keras.layers.Conv2D(512, kernel_size=3, strides=1, padding='same',activation='relu')
        self.conv4_2 = tf.keras.layers.Conv2D(512, kernel_size=3, strides=1, padding='same',activation='relu')
        self.maxpool4 = tf.keras.layers.MaxPool2D(pool_size=2)
        
        self.conv5_1 = tf.keras.layers.Conv2D(512, kernel_size=3, strides=1, padding='same',activation='relu')
        self.conv5_2 = tf.keras.layers.Conv2D(512, kernel_size=3, strides=1, padding='same',activation='relu')
        self.maxpool5 = tf.keras.layers.MaxPool2D(pool_size=2)

        self.gap = tf.keras.layers.GlobalAveragePooling2D()
        self.fc1 = tf.keras.layers.Dense(4096, activation='relu')
        self.fc2 = tf.keras.layers.Dense(4096, activation='relu')
        self.fc3 = tf.keras.layers.Dense(10, activation='softmax')        
        
    def call(self, input_tensor):
        x = self.conv1_1(input_tensor)
        x = self.maxpool1(x)
        
        x = self.conv2_1(x)
        x = self.maxpool2(x)
        
        x = self.conv3_1(x)
        x = self.conv3_2(x)
        x = self.maxpool3(x)
        
        x = self.conv4_1(x)
        x = self.conv4_2(x)
        x = self.maxpool4(x)
        
        x = self.conv5_1(x)
        x = self.conv5_2(x)
        x = self.maxpool5(x)
        
        x = self.gap(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [6]:
BATCH_SIZE = 256
EPOCH = 20

(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    as_supervised=True,
    shuffle_files=True,
    with_info=True,
)
ds_train = apply_normalize_on_dataset(ds_train, batch_size=BATCH_SIZE)
ds_test = apply_normalize_on_dataset(ds_test, batch_size=BATCH_SIZE)

In [7]:
vgg11 = VGG11()

vgg11.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_11 = vgg11.fit(
    ds_train,
    steps_per_epoch=int(ds_info.splits['train'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info.splits['test'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1
)

Epoch 1/20
195/195 [==============================] - 10s 50ms/step - loss: 2.3024 - accuracy: 0.1040 - val_loss: 2.3022 - val_accuracy: 0.0993
Epoch 2/20
195/195 [==============================] - 8s 43ms/step - loss: 2.3020 - accuracy: 0.1380 - val_loss: 2.3018 - val_accuracy: 0.1728
Epoch 3/20
195/195 [==============================] - 8s 43ms/step - loss: 2.3017 - accuracy: 0.1474 - val_loss: 2.3015 - val_accuracy: 0.1735
Epoch 4/20
195/195 [==============================] - 8s 43ms/step - loss: 2.3014 - accuracy: 0.1562 - val_loss: 2.3012 - val_accuracy: 0.1599
Epoch 5/20
195/195 [==============================] - 8s 43ms/step - loss: 2.3009 - accuracy: 0.1377 - val_loss: 2.3004 - val_accuracy: 0.1616
Epoch 6/20
195/195 [==============================] - 8s 42ms/step - loss: 2.3001 - accuracy: 0.1898 - val_loss: 2.2995 - val_accuracy: 0.2186
Epoch 7/20
195/195 [==============================] - 8s 42ms/step - loss: 2.2989 - accuracy: 0.1854 - val_loss: 2.2978 - val_accuracy: 0.170